In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import matplotlib.image as mpimg
import cv2
import h5py

In [2]:
model=tf.keras.models.load_model('facial_recognition_densenet_true_with_callbacks.h5')

In [3]:
#functions of data augmentation

def Rescale(n,size=200):
 
    x=n["x"]
    y=n["y"]
    image=n["image"] 


    x_pts=x
    y_pts=y
    
    h, w = image.shape[:2]
    
    new_h,new_w=size,size
    
    image=cv2.resize(image,(new_h,new_w))
    
    x_pts=x_pts*(new_w/w)
    y_pts=y_pts*(new_h/h)
    
    return {"image":image,"x":x_pts,"y":y_pts}




def h_flip(n):
    
    x=n["x"]
    y=n["y"]
    image=n["image"]

    h, w = image.shape[:2]
    x_pts=x
    y_pts=y
    
    image=cv2.flip(image,1)
    x_pts=w-x_pts
    y_pts=y_pts
    return {"image":image,"x":x_pts,"y":y_pts}
    
    
def v_flip(n):

    x=n["x"]
    y=n["y"]
    image=n["image"]
    
    h, w = image.shape[:2]
    x_pts=x
    y_pts=y
    
    image=cv2.flip(image,0)
    x_pts=x_pts
    y_pts=h-y_pts
    return {"image":image,"x":x_pts,"y":y_pts}


def hv_flip(n):
  i=np.random.randint(0,2)

  if(i==0):
    x=n["x"]
    y=n["y"]
    image=n["image"]
    
    h, w = image.shape[:2]
    x_pts=x
    y_pts=y
    
    image=cv2.flip(image,0)
    x_pts=x_pts
    y_pts=h-y_pts
    return {"image":image,"x":x_pts,"y":y_pts}

  else:
    x=n["x"]
    y=n["y"]
    image=n["image"]

    h, w = image.shape[:2]
    x_pts=x
    y_pts=y
    
    image=cv2.flip(image,1)
    x_pts=w-x_pts
    y_pts=y_pts
    return {"image":image,"x":x_pts,"y":y_pts}

      

    
def i_crop(n):

    x=n["x"]
    y=n["y"]
    image=n["image"]
    
    if (np.random.randint(-2,2)<0):
      return {"image":image,"x":x,"y":y}

    else:

      i=Rescale(n,size=np.random.randint(300,501))
      image1=i["image"]
      x_pts=i["x"]
      y_pts=i["y"]
      
      h, w = image1.shape[:2]
      new_h, new_w = 200,200

      top = np.random.randint(0, h - new_h)
      left = np.random.randint(0, w - new_w)

      image1 = image1[top: top + new_h,left: left + new_w]
      
      x_pts=x_pts-left
      y_pts=y_pts-top
      
      x_pts=np.fmax(0,x_pts)   #to remove the -ve values 
      y_pts=np.fmax(0,y_pts)
      
      return {"image":image1,"x":x_pts,"y":y_pts}


def h_shift(n):
    x=n["x"]
    y=n["y"]
    image=n["image"].reshape(200,200,1)
    
    h, w = image.shape[:2]
    x_shift=np.random.randint(-w/3,w/3)
    #x_shift=-34
    x_pts=x
    y_pts=y
    
    if(x_shift<=0):
        black=np.full((h,np.abs(x_shift),1),0.0)
        image1=np.hstack((image,black))
        
        image1=image1[0:h,np.abs(x_shift):np.abs(x_shift)+w]
        x_pts=np.array(x_pts)+x_shift
        y_pts=y_pts
        
        x_pts=np.fmax(0,x_pts)
        #y_pts=max(0,y_pts)
        
        return {"image":image1,"x":x_pts,"y":y_pts}
    
    else:
        black=np.full((h,np.abs(x_shift),1),0.0)
        black.shape
        image1=np.hstack((black,image))
        
        image1=image1[0:h,0:w]
        x_pts=np.array(x_pts)+x_shift
        y_pts=y_pts
        
        x_pts=np.fmin(w,x_pts)
        #y_pts=max(0,y_pts)
        
        return {"image":image1,"x":x_pts,"y":y_pts}


def v_shift(n):
    x=n["x"]
    y=n["y"]
    image=n["image"]
    
    h, w = image.shape[:2]
    y_shift=np.random.randint(-h/3,h/3)
    #x_shift=-34
    x_pts=x
    y_pts=y
    
    if(y_shift<=0): #image moves out of frmae frm top
        black=np.full((np.abs(y_shift),w,1),0.0)
        image1=np.vstack((image,black))
        
        image1=image1[np.abs(y_shift):np.abs(y_shift)+h,0:w]
        y_pts=np.array(y_pts)+y_shift
        x_pts=x_pts
        
        y_pts=np.fmax(0,y_pts)
        #y_pts=max(0,y_pts)
        
        return {"image":image1,"x":x_pts,"y":y_pts}
    
    else:
        black=np.full((np.abs(y_shift),w,1),0.0)
        black.shape
        image1=np.vstack((black,image))
        
        image1=image1[0:h,0:w]
        y_pts=np.array(y_pts)+y_shift
        x_pts=x_pts
        
        y_pts=np.fmin(h,y_pts)
        #y_pts=max(0,y_pts)
        
        return {"image":image1,"x":x_pts,"y":y_pts}


Run the below code to test the facial markers using your webcam.

In [5]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    #frame1=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    small_frame = cv2.resize(frame, (200, 200))
    bw_frame=cv2.cvtColor(small_frame,cv2.COLOR_RGB2GRAY)
    bw_frame=bw_frame/255.0
    bw1=bw_frame.reshape(1,200,200,1)
    p=model.predict(bw1)
    
    
    
    
    
    
    #cv2.rectangle(small_frame,(80,50),(150,120),(0,255,0),3)
    
    for i in range(68):
        cv2.circle(small_frame, (int(p[0,i]*100+100), int(p[0,68+i]*100+100)), radius=0, color=(0, 255, 0), thickness=3)

    
    
    
    cv2.imshow('frame',small_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


In [28]:
cap.release()
cv2.destroyAllWindows()

The code below adds goggles to the face detected in the live image

In [4]:
cap = cv2.VideoCapture(0)
sunglasses = mpimg.imread('sunglasses.png')


while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    #frame1=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    small_frame = cv2.resize(frame, (200, 200))
    bw_frame=cv2.cvtColor(small_frame,cv2.COLOR_RGB2GRAY)
    bw_frame=bw_frame/255.0
    bw1=bw_frame.reshape(1,200,200,1)
    p=model.predict(bw1)
    
    
    
    pts=p*100.0+100.0


    image_copy=np.copy(small_frame)

    

    x = int(pts[:,17])
    y = int(pts[:,68+17])

    w = int(abs(pts[:,26] - pts[:,17]))
    h = int(abs(pts[:,68+27] - pts[:,68+34]))

    new_sunglasses =  cv2.resize(sunglasses, (w, h), interpolation = cv2.INTER_CUBIC)

    new_sunglasses=new_sunglasses
    # get region of interest on the face to change
    roi_color = image_copy[y:y+h,x:x+w]

    ind = np.argwhere(new_sunglasses[:,:,3] > 0)


    for i in range(3):
        roi_color[ind[:,0],ind[:,1],i] = new_sunglasses[ind[:,0],ind[:,1],i]    
    
    # set the area of the image to the changed region with sunglasses
    image_copy[y:y+h,x:x+w] = roi_color
    
    
    #cv2.rectangle(small_frame,(80,50),(150,120),(0,255,0),3)
    
    #for i in range(68):
     #   cv2.circle(small_frame, (int(p[0,i]*100+100), int(p[0,68+i]*100+100)), radius=0, color=(0, 255, 0), thickness=3)

    
    
    
    
    
    cv2.imshow('frame',image_copy)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
